In [23]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [76]:
PV_data = pd.read_csv('PV_data.csv', header=None).to_numpy()
input_data = PV_data[0:4, :]
output_data = PV_data[4:5, :]
input_obs = input_data.shape[1]
np.random.seed(42)
random_seq = np.random.permutation(input_obs)
training_data_input = input_data[:,random_seq[0:math.floor(0.75*input_obs)]]
training_data_input_mean = np.mean(training_data_input, axis = 1, keepdims=True)
training_data_input_std = np.std(training_data_input, axis = 1, keepdims=True)
training_data_input = (training_data_input-training_data_input_mean)/training_data_input_std
training_data_output = output_data[:,random_seq[0:math.floor(0.75*input_obs)]]
dev_data_input = input_data[:,random_seq[math.floor(0.75*input_obs):]]
dev_data_input = (dev_data_input-training_data_input_mean)/training_data_input_std
dev_data_output = output_data[:,random_seq[math.floor(0.75*input_obs):]]


In [30]:
stress_pred_model = Sequential()
stress_pred_model.add(Dense(32, input_shape=(4,), activation='relu'))
stress_pred_model.add(Dense(1))

In [31]:
stress_pred_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                160       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [86]:
opt = tf.keras.optimizers.Adam(learning_rate=0.05)
stress_pred_model.compile(optimizer=opt, loss=tf.keras.losses.MeanSquaredError())

In [81]:
hist_obj = stress_pred_model.fit(training_data_input.T, training_data_output.T, epochs=5000, verbose=0)